<a href="https://colab.research.google.com/github/Subratais/genai-support-assistant/blob/main/Celonis_Customer_Support_AI_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

####Install Dependencies

In [1]:
!pip install -q sentence-transformers transformers accelerate datasets
!pip install -q chromadb fastapi uvicorn nest_asyncio pyngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 61.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 44.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

####Import Libraries

In [3]:
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.utils import embedding_functions
from datasets import load_dataset
from transformers import pipeline
import uuid
import os
import csv
from datetime import datetime
from fastapi import FastAPI
from pydantic import BaseModel
from fastapi.responses import JSONResponse
from fastapi.middleware.cors import CORSMiddleware
from chromadb.config import Settings

Secure Your Hugging Face Token

In [2]:
from huggingface_hub import login
from getpass import getpass

# Securely enter your Hugging Face Token
hf_token = getpass("Enter your Hugging Face token: ")

# Login using the token
login(token=hf_token)

Enter your Hugging Face token: ··········


####Secure your ngrok Token

In [4]:
# For security, we prompt ngrok token using getpass instead of hardcoding
#This is safer and avoids leaking credentials in shared notebooks
from getpass import getpass

#Secure token input (won't show in notebook)
authtoken = getpass("Enter your ngrok token: ")

#Add token to ngrok config
!ngrok config add-authtoken {authtoken}

Enter your ngrok token: ··········
Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


Load Dataset and Setup Vector Database

In [5]:
# Load a subset of customer support tweets from Hugging Face dataset
dataset = load_dataset("MohammadOthman/mo-customer-support-tweets-945k", split="train[:1000]")

# Extract query-response pairs from the dataset
query_response_pairs = []
for item in dataset:
    if item.get("input") and item.get("output"):
        query = item["input"].strip()
        response = item["output"].strip()
        query_response_pairs.append({"query": query, "response": response})

# Load sentence embedding model
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# Configure ChromaDB persistent storage
client_settings = Settings(persist_directory="./chroma_db")
chroma_client = chromadb.Client(settings=client_settings)

# Define the embedding function for vector search
embedding_fn = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="all-MiniLM-L6-v2")

# Create or load the ChromaDB collection
try:
    collection = chroma_client.get_collection("support_bot")
except:
    collection = chroma_client.create_collection(name="support_bot", embedding_function=embedding_fn)

# Populate the collection with embeddings if empty
if collection.count() == 0:
    for pair in query_response_pairs:
        doc_id = str(uuid.uuid4())
        collection.add(
            documents=[f"{pair['query']} {pair['response']}"],
            metadatas=[{"response": pair["response"]}],
            ids=[doc_id]
        )

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/3.40k [00:00<?, ?B/s]

preprocessed_data.json:   0%|          | 0.00/222M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/945278 [00:00<?, ? examples/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

####Load Mistral-7B

In [6]:
generator = pipeline(
    "text-generation",
    model="mistralai/Mistral-7B-Instruct-v0.1",

    device_map="auto",
    model_kwargs={"torch_dtype": "auto"}
)

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

Device set to use cuda:0


FastAPI Service for LLM Response Generation and Evaluation Logging

In [14]:
from fastapi import FastAPI
from pydantic import BaseModel
from fastapi.responses import JSONResponse
from fastapi.middleware.cors import CORSMiddleware
from datetime import datetime
from typing import Optional
import csv
import re

app = FastAPI()

# Allow API access from browser / Swagger / external frontend
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_methods=["*"],
    allow_headers=["*"]
)

# Request schema for generating LLM-based response
class QueryInput(BaseModel):
    query: str

# Request schema for submitting manual scores
class ScoreInput(BaseModel):
    query: str
    answer: str
    top_document: str
    relevance: int
    correctness: int
    conciseness: int
    fluency: int

# Endpoint: Generate model response and auto-score with LLM-as-judge
@app.post("/generate_response")
async def generate_response(payload: QueryInput):
    user_query = payload.query

    # Vector-based document retrieval (top 3)
    results = collection.query(query_texts=[user_query], n_results=3)
    retrieved_docs = results["documents"][0]
    top_doc = retrieved_docs[0]
    context = "\n\n".join(retrieved_docs)

    # Main prompt for response generation
    prompt = f"""<s>[INST] <<SYS>>
You are a customer support agent. Answer in 2-3 short, polite sentences.
Avoid repetition and ask for clarification if needed.
<</SYS>>

Context:
{context}

User Query:
{user_query}

Give a direct, non-repeating answer. [/INST]"""

    # Generate model output
    try:
        generated = generator(
            prompt,
            max_new_tokens=100,
            temperature=0.2,
            do_sample=True,
            repetition_penalty=2.0,
            no_repeat_ngram_size=3,
            early_stopping=True
        )[0]["generated_text"]
    except Exception as e:
        return JSONResponse(status_code=500, content={"error": str(e)})

    # Extract the generated answer from the output
    answer = generated.split("[/INST]")[-1].strip()
    if answer.count(".") > 2:
        answer = answer.split(".")[0] + "."

    # Create evaluation prompt (must match regex expectation)
    judge_prompt = f"""<s>[INST] <<SYS>>
You are an expert grader. Strictly rate the chatbot response on 4 parameters:

Return scores in this exact format (only numbers):
Relevance: [1-5]
Correctness: [1-5]
Conciseness: [1-5]
Fluency: [1-5]

Do NOT explain anything. Just output the scores exactly like above.
<</SYS>>

Query: {user_query}

Context:
{context}

Response:
{answer}
[/INST]"""

    # LLM evaluates its own answer
    try:
        judge_response = generator(
            judge_prompt,
            max_new_tokens=100,
            temperature=0
        )[0]["generated_text"]
    except Exception as e:
        judge_response = ""
        print("Judge model failed:", e)

    # Extract scores using strict pattern matching
    match = re.search(
        r"Relevance:\s*(\d+)\s*Correctness:\s*(\d+)\s*Conciseness:\s*(\d+)\s*Fluency:\s*(\d+)",
        judge_response
    )
    if match:
        relevance, correctness, conciseness, fluency = match.groups()
    else:
        relevance = correctness = conciseness = fluency = "-"

    # Save evaluation log to CSV
    with open("response_eval_log.csv", mode="a", newline="") as file:
        writer = csv.writer(file)
        writer.writerow([
            datetime.now(), user_query, answer, top_doc,
            relevance, correctness, conciseness, fluency, "auto"
        ])

    # Return structured output
    return JSONResponse(content={
        "query": user_query,
        "answer": answer,
        "top_document": top_doc,
        "context": context,
        "evaluation_source": "auto",
        "evaluation": {
            "relevance": relevance,
            "correctness": correctness,
            "conciseness": conciseness,
            "fluency": fluency
        }
    })


# Endpoint: Submit manual scores (after human review)
@app.post("/submit_scores")
async def submit_manual_scores(payload: ScoreInput):
    try:
        with open("response_eval_log.csv", mode="a", newline="") as file:
            writer = csv.writer(file)
            writer.writerow([
                datetime.now(), payload.query, payload.answer, payload.top_document,
                payload.relevance, payload.correctness,
                payload.conciseness, payload.fluency, "manual"
            ])
        return {"status": "success", "message": "Manual scores saved successfully"}
    except Exception as e:
        return JSONResponse(status_code=500, content={"error": str(e)})


Expose FastAPI API on Public URL

In [15]:
from pyngrok import ngrok
import nest_asyncio
import uvicorn

public_url = ngrok.connect(8000)
print("🚀 Public URL:", public_url)

nest_asyncio.apply()
uvicorn.run(app, host="0.0.0.0", port=8000)

🚀 Public URL: NgrokTunnel: "https://3e29-34-125-50-93.ngrok-free.app" -> "http://localhost:8000"


INFO:     Started server process [349]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     163.116.214.57:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     163.116.214.57:0 - "GET /openapi.json HTTP/1.1" 200 OK


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


INFO:     163.116.214.57:0 - "POST /generate_response HTTP/1.1" 200 OK


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


INFO:     163.116.214.57:0 - "POST /generate_response HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [349]


Manually Check response_eval_log.csv

In [16]:
import pandas as pd

df = pd.read_csv("response_eval_log.csv", header=None)
df.columns = ["timestamp", "query", "answer", "top_doc", "relevance", "correctness", "conciseness", "fluency", "source"]
df.tail(5)


,timestamp,query,answer,top_doc,relevance,correctness,conciseness,fluency,source
1,2025-04-20 13:40:31.297637,Why did you suggest contacting support?,To resolve an issue or concern related,Tell me why I have called support four times a...,4,3,4,3,auto
2,2025-04-20 13:49:09.446915,Why did you suggest contacting support?,Thank you for reaching out through our website...,Tell me why I have called support four times a...,-,-,-,-,auto
3,2025-04-20 13:49:53.859777,Why did you suggest contacting support?,To resolve an issue or concern related,Tell me why I have called support four times a...,-,-,-,-,auto
4,2025-04-20 13:53:02.605452,Why did you suggest contacting support?,To resolve an issue or concern related,Tell me why I have called support four times a...,4,3,4,3,auto
5,2025-04-20 13:54:09.539888,Why did you suggest contacting support?,The user contacted our company's technical or ...,Tell me why I have called support four times a...,4,4,3,4,auto
